# Import libraries and set plot dimensions

In [3]:
# libraries
library(tidyverse)
library(edgeR)
library(sva)
library(reshape2)
library(pheatmap)
library(RColorBrewer)
library(MetBrewer)
library(caret)
library(glmnet)
library(e1071)
library(edgeR)
library(umap)
library(DMwR)
library(pROC)
library(gbm)
library(survival)
library(survminer)
library(GenomicRanges)
library(recipes)

In [ ]:
# plot dimensions
options(repr.plot.width = 10, repr.plot.height = 10) # specific to the IRkernel

# Data pre-processing + differential analysis (edgeR)

## Import counts data and compile into a numeric matrix

In [ ]:
# import 10kb reference, w regulatory features and bin IDs. Reference was created using annotatR. Note; reference generation requires an internet connection.
setwd("/path/to/reference/")
annotated_ref_no_X_Y_M_blacklist <- read.delim("10kb_bin_genome_wide_annotated_reference_with_regulatory_features_no_X_Y_M_ENCODEBlacklist.txt", header = TRUE, sep = "\t")

# import sample list, formatted beforehand.
setwd("/path/to/list/")
sample_list <- read.delim("sample_list_for_lymphoma_cohort.txt", header = TRUE, sep = "\t")

gc() #clear memory if memory is a bottleneck; the data frames in this script are large

# import non-normalized counts, output from MEDIPS (in 10kb bins)
# BAM files prior to generation of non-normalized counts were generated using MEDIPIPE pre-processing pipeline (10.1093/bioinformatics/btad423, originally intended for cfMeDIP-Seq data, but adapted for cfChIP-Seq). 
setwd("/path/to/samples/")
import_counts <- list.files(path = ".", pattern = "counts_10kb.txt$", recursive = TRUE) # list counts with directory paths

# exclude samples with these names, as they are not required for the following analyses. Only T1 (baseline) samples should be included.
exclude <- c("0018_T3", "0025_T4", "0027_T3", "0027_T4", "0101_T3", "0030_T1", "HUCON_37", "HUCON_43", "HUCON_44", "0036_C", "0247_T1_ABC", "T3", "T5", "pbmc_k27") #character vector of sample strings to be excluded.

# convert character vector to data frame
import_counts <- as.data.frame(import_counts)

# Remove all excluded samples from the original list of samples. Use "|" as the seperator between vector elements.
import_counts <- import_counts[!grepl(paste(exclude, collapse = "|"), import_counts$import_counts),] # 

# loop over files in import_counts, and read them into memory
counts <- mapply(read.delim, import_counts) 

# bind individual datasets across columns
counts_matrix <- do.call(cbind, counts)

# change column names to file names from list, and remove NAs
colnames(counts_matrix) <- c(paste0(sample_list$sample_name))
counts_matrix <- na.omit(counts_matrix)

# index rows in counts_matrix
rownames(counts_matrix) <- 1:nrow(counts_matrix)

gc()

## Data cleaning, filter out low variance and low count features from the matrix

In [ ]:
#filter out chrX, Y, and ENCODE Blacklist regions. Resulting data frame is your ground truth; refer back to this after feature selection.
all_filtered_features <- counts_matrix[as.integer(rownames(counts_matrix)) %in% annotated_ref_no_X_Y_M_blacklist$bin_id,]

# calculate the coefficient of variation (CV, ratio of SD to the mean) for all features using the var function, row-wise. Independent of sample origin.
cv_features <- apply(all_filtered_features, 1, var)

# visualize histogram of feature variances
var_hist <- hist(cv_features[cv_features > 0 & cv_features < 40000], breaks = 500) # bimodal distribution, left skewed

# remove variance below the valley on the histogram. Valley is at around 2400.
all_filtered_features <- all_filtered_features[(which(apply(all_filtered_features, 1, var)>=2400)),]

# remove low count features
counts_mean <- apply(all_filtered_features, 1, mean) # left skewed, slight bimodal distribution of counts after filtering low variance features.
counts_hist <- hist(counts_mean[counts_mean > 0 & counts_mean < 1000], breaks = 500) 

# Remove lowest expressed features
all_filtered_features <- all_filtered_features[(which(apply(all_filtered_features, 1, mean)>=50)),]

gc() #clear memory

## Split dataset into train and test sets for feature selection / validation

In [ ]:
# split train and test sets. Perform differential analysis on only training set.

# Split the data into features (X) and labels (y)
X <- all_filtered_features  # Features (numeric matrix plus sample name attributes)
y <- sample_list$timepoint # Data labels for lymphoma vs non-lymphoma classification

# Training: 70%; Test: 30%
# Split the data into training and testing sets
set.seed(23)  # For reproducibility

train_indices <- createDataPartition(y, p = 0.7, list = FALSE)

# save train and test set numeric matrices and labels as independent variables. Lock test set away for later validation.
X_train <- X[, train_indices]
y_train <- y[train_indices]
X_test <- X[, -train_indices]
y_test <- y[-train_indices]

#check balance of healthy and baseline samples in train/test splits. Ensure it is close to 70/30.
table(y_train)
table(y_test)

## Perform differential analysis using edgeR, using SVA to address unknown patterns in the data

In [ ]:
# this chunk takes time and computational power, so schedule script as a job if working on a virtual machine.
# script for differential analysis was adapted from edgeR user guide (https://bioconductor.org/packages/release/bioc/vignettes/edgeR/inst/doc/edgeRUsersGuide.pdf)
gc()

# set up DGEList object for differential analysis
design <- model.matrix(~y_train) #design model around differences between lymphoma and non-lymphoma profiles. Can change depending on intended comparison.
list <- DGEList(counts = X_train, samples = y_train, remove.zeros = FALSE) # no need to remove zeros, since they were removed in matrix pre-processing steps.

# apply TMM normalization to raw counts
tmm_norm <- calcNormFactors(list, method="TMM")

# normalize counts before SVA is performed; otherwise library size is the only surrogate variable.
cpm_norm <- cpm(tmm_norm, normalized.lib.sizes = TRUE, log = FALSE)

# perform SVA analysis to remove technical variation
sva_fit <- sva(cpm_norm, design)

#IMPORTANT; add SVA surrogate variables as covariates in model design.
design <- model.matrix(~y_train + sva_fit$sv)

## save model before differential analysis is applied, in case the analysis is not complete within the bounds of the wall time allocated.
#save.image()

gc()

# steps below here are to perform the differential analysis
fit <- estimateDisp(tmm_norm, design = design, trend.method = "locfit", robust = TRUE)

gc()

# fit model, incorporating surrogate variables
QL <- glmQLFit(fit, design = design)

#Subset to rows with an FDR <0.05
fdr <- table(p.adjust(QL$table$PValue, method="BH")<0.05)

lrt <- glmQLFTest(QL, coef = 2)

# extract all features, sorted by p-val
res <- topTags(lrt, n = 246275, adjust.method = "BH", sort.by = "PValue")

# save dataset
setwd("path/to/working/directory/")
save.image("train_test_split_for_differential_analysis.RData")

## Subset to significant features from differential analysis, generate volcano plot of features as sanity check

In [ ]:
setwd("path/to/working/directory/")
load("train_test_split_for_differential_analysis.RData")

#keep only the features that meet a p-value threshold.
#de_features_subset <- res[(res$table$logFC >= 1 | res$table$logFC <= -1) & res$table$PValue <= 0.01,] # logFC >= 1 is 2x change; only 19 features.
#de_features_subset <- res[(res$table$logFC >= 0.25 | res$table$logFC <= -0.25) & res$table$PValue <= 0.05,] # 341 features.
#de_features_subset <- res[res$table$PValue <= 0.01,] # 1742 features
de_features_subset <- res[res$table$PValue <= 0.05,] # 9370 features; stick with this one as it is less stringent. Let the Random Forest decide which features are important.

# store feature names (aka bin IDs) for differential features and all features as separate variables
de_features <- rownames(de_features_subset)
de_features_all <- rownames(res)

# calculate logFC, pval, and -log10(pval) for all features
logFC <- res$table$logFC
p_value <- res$table$PValue
neg_log_p_value <- -log10(p_value)

# compile data for volcano plot.
df_feature_stats <- data.frame(logFC,neg_log_p_value,de_features_all)

# calculate logFC, pval, and -log10(pval) for all significant features
logFC_signif <- de_features_subset$table$logFC
p_value_signif <- de_features_subset$table$PValue
neg_log_p_value_signif <- -log10(p_value_signif)

# compile data for volcano plot.
df_feature_stats_signif <- data.frame(logFC_signif,neg_log_p_value_signif,de_features)

v <- ggplot() +
  geom_point(data = df_feature_stats, aes(x = logFC, y = neg_log_p_value), colour = "black", alpha = 0.3) +
  geom_point(data = df_feature_stats_signif, aes(x = logFC_signif, y = neg_log_p_value_signif), colour = "red", alpha = 0.3) +
  geom_hline(yintercept = -log10(0.05), linetype = "dashed", color = "red") +
  geom_vline(xintercept = c(-1, 1), linetype = "dashed", color = "blue") +
  labs(x = "Log Fold Change", y = "-log10(p-value)") +
  ggtitle("Volcano Plot") +
  theme_minimal()
print(v)

## Perform Principal Component Analysis (PCA) on all significant differential features

In [ ]:
# plot PCA of significant features, to see whether the features differentiate between lymphoma profiles and non-lymphoma controls
# compare to PCA of all features

# ensure you transpose the feature matrix before PCA
# pca_data <- t(cpm_norm) # for all features
pca_data <- t(cpm_norm[de_features,]) # for feature subset

# perform PCA, with z-scaling
pca_res <- prcomp(pca_data, scale = TRUE)

# Plot the first two principal components. Commented geoms are additional data layers, for additional visualizations.
q <- pca_res$x %>% 
  as.data.frame %>%
  ggplot(aes(x=PC1,y=PC2)) +
  #geom_point(aes(color=list$samples$DIAGNOSIS_CLASS_SUMMARIZE_HEALTHY),size=5) +
  #geom_point(aes(color=list$samples$DIAGNOSIS_CLASS_SUMMARIZE_HEALTHY.DIAGNOSIS_SPECIFIC_DLBCL),size=5) +
  #geom_point(aes(color=list$samples$timepoint),size=5) +
  geom_point(aes(color=y_train),size=5) +
  #geom_point(aes(color=list$samples$batch),size=5) +
  #geom_text(aes(label=list$samples$sample_name)) +
  theme_minimal(base_size=20) + 
  labs(colour = "Group") +
  xlab(paste0("PC1 (", round(pca_res$sdev[1]^2*100/sum(pca_res$sdev^2), 1), "%)")) +
  ylab(paste0("PC2 (", round(pca_res$sdev[2]^2*100/sum(pca_res$sdev^2), 1), "%)")) +
  theme(legend.position="right")

print(q)

# Random Forest of significant differential features (additional feature selection step)

In [ ]:
# chunk requires at least 60GB of memory on a virtual machine, run script as a scheduled job on compute cluster, requesting compute node.

# import dataset with significant differential features
setwd("path/to/working/directory/")
load("train_test_split_for_differential_analysis.RData")

# subset differential results to differential features
de_features_subset <- res[res$table$PValue <= 0.05,] # 9370 features
# de_features_subset <- res[res$table$PValue <= 0.05 & res$table$logFC > 0,] # 5328 features; only hypermethylated features considered. Compare classification performance

# save bin IDs (aka rownames) of differential features, depending on which set you're using
de_features <- rownames(de_features_subset)

# subset CPM normalized matrix to differential features
de_features_matrix <- cpm_norm[de_features,]

# specify tune grid for hyperparameter tuning; tuning hyperparameters depend on the model used.
tune_grid <- expand.grid(
  mtry = c(2, 5, 10) # tuning hyperparameter for rf. Can test a range of values here, or do a grid search in trainControl
)

# set up train control for Random Forest model (R Caret).
ctrl <- trainControl(
  method = "repeatedcv",  # Use cross-validation for evaluation
  repeats = 5, # number of model iterations
  number = 10,  # Number of folds for cross-validation
  search = "grid",
  sampling = "smote",
  verbose = FALSE
)

# transpose feature matrix for machine learning
t_de_features_matrix <- t(de_features_matrix)

model <- train(t_de_features_matrix, 
               as.factor(y_train), 
               method = "rf", # for Random Forest
               # method = "nb", # naive bayes; turn off tuneGrid.
               # method = "glmnet", # try with specifying the tuneLength parameter next
               trControl = ctrl, 
               tuneGrid = tune_grid, # turn off for nb
               metric = "Accuracy",
               #metric = "ROC",
               #metric = "Kappa", # for unbalanced sets; since we're using SMOTE, accuracy should be okay
               ntree = 500 # for rf
               #tuneLength = 8 # take random values for default tuning parameters and pick the best; alternative to tuneGrid
)

# print error log for model output, in case the model fails
# sink(rf_hyperparameter_tuning_accuracy_error_log.txt)

# see model results for training data
print(model)

# extract predictors for further analysis (aka feature validation)
predictors <- predictors(model)

# save dataset w model
setwd("path/to/working/directory/")
save.image("rf_model_from_differential_features.RData")